# Getting started with MovingPandas

<img align="right" src="https://movingpandas.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/movingpandas/movingpandas-examples/main?filepath=1-tutorials/1-getting-started.ipynb)
[![IPYNB](https://img.shields.io/badge/view-ipynb-hotpink)](https://github.com/movingpandas/movingpandas-examples/blob/main/1-tutorials/1-getting-started.ipynb)
[![HTML](https://img.shields.io/badge/view-html-green)](https://movingpandas.github.io/movingpandas-website/1-tutorials/1-getting-started.html)

MovingPandas provides a trajectory datatype based on GeoPandas.
The project home is at https://github.com/movingpandas/movingpandas

The documentation is available at https://movingpandas.readthedocs.io/

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import hvplot.pandas 

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts

import warnings
warnings.filterwarnings('ignore')

opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=500, frame_height=400))

mpd.show_versions()

## Creating a trajectory from scratch

Trajectory objects consist of a trajectory ID and a GeoPandas GeoDataFrame with a DatetimeIndex. The data frame therefore represents the trajectory data as a Pandas time series with associated point locations (and optional further attributes).

Let's create a small toy trajectory to see how this works:

In [ ]:
df = pd.DataFrame([
  {'geometry':Point(0,0), 't':datetime(2018,1,1,12,0,0)},
  {'geometry':Point(6,0), 't':datetime(2018,1,1,12,6,0)},
  {'geometry':Point(6,6), 't':datetime(2018,1,1,12,10,0)},
  {'geometry':Point(9,9), 't':datetime(2018,1,1,12,15,0)}
]).set_index('t')
gdf = GeoDataFrame(df, crs=31256)
toy_traj = mpd.Trajectory(gdf, 1)
toy_traj

In [ ]:
toy_traj.plot()

In [ ]:
toy_traj.hvplot()

We can also access the trajectory's GeoDataFrame:

In [ ]:
toy_traj.df

In [ ]:
toy_traj.df.plot()

## Loading trajectories from CSV

In [ ]:
df = pd.read_csv('../data/geolife_small.csv', delimiter=';')
df

In [ ]:
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
print(traj_collection)

In [ ]:
traj_collection.plot(column='trajectory_id', legend=True, figsize=(9,5))

## Loading trajectories from a GeoPackage

In [ ]:
gdf = read_file('../data/geolife_small.gpkg')
gdf

After reading the trajectory point data from file, we want to construct the trajectories.

In [ ]:
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id', t='t')
print(traj_collection)

In [ ]:
traj_collection.plot(column='trajectory_id', legend=True, figsize=(9,5))

In [ ]:
traj_collection.plot()

### Accessing individual trajectories

In [ ]:
my_traj = traj_collection.trajectories[1]
print(my_traj)

In [ ]:
my_traj.plot(linewidth=5, capstyle='round', figsize=(9,3))

To visualize trajectories in their geographical context, we can also create interactive plots with basemaps:

In [ ]:
my_traj.hvplot(line_width=7.0, tiles='OSM')

## Creating a TrajectoryCollection from scratch

In [ ]:
df = pd.DataFrame(
    {
        "t": pd.date_range("2020-01-01", periods=5, freq="min"),
        "trajectory_id": [1, 1, 2, 2, 2],
        "geometry": [Point(0, 0), Point(0, 1), Point(1, 2), Point(1, 3), Point(2, 4)],
    }
)
gdf = gpd.GeoDataFrame(df, crs=4326)
tc = mpd.TrajectoryCollection(gdf, traj_id_col='trajectory_id', t='t')
tc

In [ ]:
tc.hvplot()

In [ ]:
min_duration = timedelta(minutes=1)
tc.trajectories = [traj for traj in tc if traj.get_duration() > min_duration]
tc